In [ ]:
#call library

import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

#from preprocesslib import preprocessEach, makeY

In [ ]:
import os
import pandas as pd
import numpy as np

'''
이런식으로 사용하면 됩니다.
from preprocesslib import preprocessEach, makeY
X = preprocessEach('overlap')
X bootstrap해서 -> X_final만든 후에
y = makeY('overlap', len(X_final))
'''

def preprocessEach(folder_name):
   # set file path
    # folder_path = f'/content/drive/MyDrive/research/data/{folder_name}_clean'
    folder_path = f'../data/{folder_name}_clean'


    # load file name in folder
    file_names = os.listdir(folder_path)
    data_array = []

    for file in file_names:
        # set file path
        file_path = os.path.join(folder_path, file)
        
        # load csv file as dataframe
        df = pd.read_csv(file_path)

        #preprocessing column
        df['open-close'] = df['Open'] - df['Close'] #추가
        df['open-close'] = df['open-close'] + abs(min(df['open-close'])) #추가
        df['MA5'] = df['Close'].rolling(5).mean() #추가
        df['Diff'] = df['High'] - df['Low']
        df = df[['Close','Volume','Diff','open-close','MA5', 'Search']]
        # df = df[['Search']]
        df.fillna(0, inplace=True)

        #log transform
        df = np.log1p(df)

        #insert list to array
        data_array.append(df)

    return np.array(data_array)

def makeY(folder_name, filelength):

    if folder_name=='growth':
        yGrowths = np.ones(filelength)
        yValues = np.zeros(filelength)
    elif folder_name=='value':
        yGrowths = np.zeros(filelength)
        yValues = np.ones(filelength)
    elif folder_name=='overlap':
        yGrowths = np.ones(filelength)
        yValues = np.ones(filelength)

    y = np.column_stack([yGrowths.T, yValues.T])
    return y

In [ ]:
xGrowths = preprocessEach('growth')
xValues = preprocessEach('value')
xOverlaps = preprocessEach('overlap')

In [ ]:
#bootstrap
n_samples = len(xValues)
diff = len(xValues) - len(xGrowths)
xGrowths_bootstrapped = xGrowths[np.random.choice(len(xGrowths), size=diff, replace=True)]
xGrowths = np.concatenate((xGrowths, xGrowths_bootstrapped), axis=0)

diff = len(xValues) - len(xOverlaps)
xOverlaps_bootstrapped = xOverlaps[np.random.choice(len(xOverlaps), size=diff, replace=True)]
xOverlaps = np.concatenate((xOverlaps, xOverlaps_bootstrapped), axis=0)

In [ ]:
#make y values
yGrowths = makeY('growth', n_samples)
yValues = makeY('value', n_samples)
yOverlaps = makeY('overlap', n_samples)

In [ ]:
x = np.concatenate((xGrowths, xValues, xOverlaps), axis=0)
y = np.concatenate((yGrowths, yValues, yOverlaps), axis=0)

In [ ]:
#minmax scaling by element
scaler = MinMaxScaler()
for i in range(6):
    x[:,:,i] = scaler.fit_transform(x[:,:,i])

In [ ]:
def splitData(x, y, train_ratio=0.8):
    # shuffle data
    np.random.seed(2023)
    permutation = np.random.permutation(len(x))
    x = x[permutation]
    y = y[permutation]

    # split data
    n_train_samples = int(train_ratio * len(x))
    x_train, x_test = x[:n_train_samples], x[n_train_samples:]
    y_train, y_test = y[:n_train_samples], y[n_train_samples:]

    return x_train, x_test, y_train, y_test

In [ ]:
#cross validation
X_train, X_test, Y_train, Y_test = splitData(x,y)

In [ ]:
growth_Y_test = Y_test[:,0]
value_Y_test = Y_test[:,1]

In [ ]:
unit = 32
batch = 128
dropout = 0.4
epoch = 2000
model,es,mc = makeModel(unit, dropout)
history=model.fit(X_train, Y_train, epochs=epoch, batch_size=batch, validation_split=0.2, callbacks=[es, mc])
model=load_model('best_model.h5')
loss, accuracy = model.evaluate(X_test, Y_test)

In [ ]:
value_model=load_model('./LSTM/best_model_value.h5')
growth_model = load_model('./LSTM/best_model_growth.h5')

value_pred = value_model.predict(X_test)
growth_pred = growth_model.predict(X_test)
Y_pred_per = np.concatenate([growth_pred,value_pred], axis=1)


In [ ]:
def predictLabelForGivenThreshold(threshold):
  y_pred=[]
  for sample in  Y_pred_per:
    y_pred.append([1 if i>=threshold else 0 for i in sample ] )
  return np.array(y_pred)

In [ ]:
def accuracy(y_true, y_pred):
    
    """
    Function to calculate accuracy
    -> param y_true: list of true values
    -> param y_pred: list of predicted values
    -> return: accuracy score
    
    """
    
    # Intitializing variable to store count of correctly predicted classes
    correct_predictions = 0
    
    for yt, yp in zip(y_true, y_pred):
        
        if yt == yp:
            
            correct_predictions += 1
    
    #returns accuracy
    return correct_predictions / len(y_true)

In [ ]:
# value_n_pred = np.where(value_pred >= .5, 1, 0)
# growth_n_pred = np.where(growth_pred >= .5, 1, 0)

Y_pred = predictLabelForGivenThreshold(0.5)


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, Y_pred))

In [ ]:
from sklearn.metrics import classification_report
label_names = ['Growth', 'Value']
print(classification_report(Y_test, Y_pred, target_names=label_names))


In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(Y_test, Y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


In [ ]:
def y_bin(selectedLabel):
    y_binary = Y_test[:,selectedLabel]
    y_binary_scores = Y_pred_per[:,selectedLabel]
    return y_binary, y_binary_scores

In [ ]:
import matplotlib.pyplot as plt

def plt_roc_curve(fpr, tpr, selectedVal):
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve' )
    plt.plot([0, 1], [0, 1], color='navy',linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for {}'.format(label_names[selectedVal]))
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve
growth_val = 0
growth_y_bin, growth_y_bin_per = y_bin(growth_val)

g_fpr, g_tpr, g_thresholds = roc_curve(growth_y_bin, growth_y_bin_per)
plt_roc_curve(g_fpr, g_tpr, growth_val)

In [ ]:
value_val = 1
value_y_bin, value_y_bin_per = y_bin(value_val)
v_fpr, v_tpr, v_thresholds = roc_curve(value_y_bin, value_y_bin_per)
plt_roc_curve(v_fpr, v_tpr, value_val)